## Example: Transitive Closure

This example shows how to compute transitive closure using
recursive rules and stratification.

In [2]:
from rdflib import Graph, Namespace
from srl.parser import SRLParser
from srl.engine import RuleEngine

# Define namespace
EX = Namespace("http://example.org/")

# Create graph with parent relationships
graph = Graph()
graph.bind("ex", EX)

graph.add((EX.Alice, EX.parent, EX.Bob))
graph.add((EX.Bob, EX.parent, EX.Charlie))
graph.add((EX.Charlie, EX.parent, EX.Diana))

print("Input data (parent relationships):")
for s, p, o in graph:
    print(f"  {s.n3(graph.namespace_manager)} {p.n3(graph.namespace_manager)} {o.n3(graph.namespace_manager)}")

# Define recursive rules for transitive closure
rule_text = """
PREFIX ex: <http://example.org/>

# Base case: parent is ancestor
RULE {
    ?x ex:ancestor ?y .
} WHERE {
    ?x ex:parent ?y .
}

# Recursive case: ancestor of ancestor
RULE {
    ?x ex:ancestor ?z .
} WHERE {
    ?x ex:ancestor ?y .
    ?y ex:ancestor ?z .
}
"""

# Parse and evaluate
parser = SRLParser()
rule_set = parser.parse(rule_text)

engine = RuleEngine(rule_set)
result_graph = engine.evaluate(graph, inplace=False)

# Show all ancestor relationships
print("\nInferred ancestor relationships:")
for s, p, o in sorted(result_graph):
    if p == EX.ancestor:
        print(f"  {s.n3(result_graph.namespace_manager)} {p.n3(result_graph.namespace_manager)} {o.n3(result_graph.namespace_manager)}")

print(f"\nTotal: {len(graph)} input triples → {len(result_graph)} output triples")
print("Transitive closure computed successfully!")


Input data (parent relationships):
  ex:Charlie ex:parent ex:Diana
  ex:Bob ex:parent ex:Charlie
  ex:Alice ex:parent ex:Bob

Inferred ancestor relationships:
  <http://example.org/Alice> <http://example.org/ancestor> <http://example.org/Bob>
  <http://example.org/Alice> <http://example.org/ancestor> <http://example.org/Charlie>
  <http://example.org/Alice> <http://example.org/ancestor> <http://example.org/Diana>
  <http://example.org/Bob> <http://example.org/ancestor> <http://example.org/Charlie>
  <http://example.org/Bob> <http://example.org/ancestor> <http://example.org/Diana>
  <http://example.org/Charlie> <http://example.org/ancestor> <http://example.org/Diana>

Total: 3 input triples → 9 output triples
Transitive closure computed successfully!


## Example: BIND and String Operations

This example shows using BIND to create new variables
with computed values, including string concatenation.

In [ ]:
from rdflib import Graph, Namespace, Literal
from srl.parser import SRLParser
from srl.engine import RuleEngine

# Define namespace
EX = Namespace("http://example.org/")

# Create graph with name information
graph = Graph()
graph.bind("ex", EX)

graph.add((EX.Person1, EX.firstName, Literal("John")))
graph.add((EX.Person1, EX.lastName, Literal("Doe")))
graph.add((EX.Person2, EX.firstName, Literal("Jane")))
graph.add((EX.Person2, EX.lastName, Literal("Smith")))

print("Input data:")
for s, p, o in sorted(graph):
    print(f"  {s.n3(graph.namespace_manager)} {p.n3(graph.namespace_manager)} {o}")

# Define rule with BIND and CONCAT
rule_text = """
PREFIX ex: <http://example.org/>

RULE {
    ?person ex:fullName ?fullName .
} WHERE {
    ?person ex:firstName ?first .
    ?person ex:lastName ?last .
    BIND(CONCAT(?first, " ", ?last) AS ?fullName)
}
"""

# Parse and evaluate
parser = SRLParser()
rule_set = parser.parse(rule_text)

engine = RuleEngine(rule_set)
result_graph = engine.evaluate(graph, inplace=False)

# Show generated full names
print("\nGenerated full names:")
for s, p, o in sorted(result_graph):
    if p == EX.fullName:
        print(f"  {s.n3(result_graph.namespace_manager)} -> {o}")

print("BIND successfully created new variables with CONCAT")


Input data:
  ex:Person1 ex:firstName John
  ex:Person1 ex:lastName Doe
  ex:Person2 ex:firstName Jane
  ex:Person2 ex:lastName Smith

Generated full names:
  <http://example.org/Person1> -> John Doe
  <http://example.org/Person2> -> Jane Smith

BIND successfully created new variables with CONCAT


In [6]:
from rdflib import Graph, Namespace, Literal
from srl.parser import SRLParser
from srl.engine import RuleEngine

# Define namespace
EX = Namespace("http://example.org/")

# Create graph with name information
graph = Graph()
graph.bind("ex", EX)

graph.add((EX.Alice, EX.parentOf,EX.Bob))
graph.add((EX.Bob, EX.parentOf,EX.Carol))
graph.add((EX.Carol, EX.parentOf,EX.Dave))

print("Input data:")
for s, p, o in sorted(graph):
    print(f"  {s.n3(graph.namespace_manager)} {p.n3(graph.namespace_manager)} {o.n3(graph.namespace_manager)}")

# Define rule with BIND and CONCAT
rule_text = """
PREFIX ex: <http://example.org/>

# Find grandchildren using path sequence (parent/parent)
RULE { ?grandparent ex:grandchildOf ?grandchild } WHERE {
    ?grandchild ex:parentOf/ex:parentOf ?grandparent
}

# Find great-grandparents
RULE { ?person ex:greatGrandparent ?ggp } WHERE {
    ?person ex:parentOf/ex:parentOf/ex:parentOf ?ggp
}
"""

# Parse and evaluate
parser = SRLParser()
rule_set = parser.parse(rule_text)

engine = RuleEngine(rule_set)
result_graph = engine.evaluate(graph)

# Show generated full names
print("\nGenerated full names:")
for s, p, o in sorted(result_graph):
    
     print(f"  {s.n3(graph.namespace_manager)} {p.n3(graph.namespace_manager)} {o.n3(graph.namespace_manager)}")


Input data:
  ex:Alice ex:parentOf ex:Bob
  ex:Bob ex:parentOf ex:Carol
  ex:Carol ex:parentOf ex:Dave

Generated full names:
  ex:Alice ex:greatGrandparent ex:Bob
  ex:Alice ex:parentOf ex:Bob
  ex:Bob ex:grandchildOf ex:Alice
  ex:Bob ex:greatGrandparent ex:Carol
  ex:Bob ex:parentOf ex:Carol
  ex:Carol ex:grandchildOf ex:Bob
  ex:Carol ex:greatGrandparent ex:Dave
  ex:Carol ex:parentOf ex:Dave
  ex:Dave ex:grandchildOf ex:Carol

Generated full names:
  ex:Alice ex:greatGrandparent ex:Bob
  ex:Alice ex:parentOf ex:Bob
  ex:Bob ex:grandchildOf ex:Alice
  ex:Bob ex:greatGrandparent ex:Carol
  ex:Bob ex:parentOf ex:Carol
  ex:Carol ex:grandchildOf ex:Bob
  ex:Carol ex:greatGrandparent ex:Dave
  ex:Carol ex:parentOf ex:Dave
  ex:Dave ex:grandchildOf ex:Carol


In [7]:
rule_set

RuleSet(prologue=Prologue(base=None, prefixes={Token('PNAME_NS', 'ex:'): IRI(value='http://example.org/')}, version=None, imports=[]), rules=[Rule(head=RuleHead(templates=[TripleTemplate(subject=Variable(name='grandparent'), predicate=IRI(value='http://example.org/grandchildOf'), object=Variable(name='grandchild'))]), body=RuleBody(elements=[TriplePattern(subject=Variable(name='grandchild'), predicate=IRI(value='http://example.org/parentOf'), object=Variable(name='grandparent'))]), layer=None, depends_on=[]), Rule(head=RuleHead(templates=[TripleTemplate(subject=Variable(name='person'), predicate=IRI(value='http://example.org/greatGrandparent'), object=Variable(name='ggp'))]), body=RuleBody(elements=[TriplePattern(subject=Variable(name='person'), predicate=IRI(value='http://example.org/parentOf'), object=Variable(name='ggp'))]), layer=None, depends_on=[])], data_blocks=[], layers=None)